# Analysis of Customer Transactions

In [ ]:
import io
import os
import requests
import zipfile

from pathlib import Path

import geopandas as gpd
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import numpy as np
import pandas as pd
import seaborn as sns

from fin_crime.data_processor import DataProcessor

In [ ]:
idx = pd.IndexSlice

pd.options.display.max_columns = 100
pd.set_option('display.float_format', lambda x: '%.5f' % x)

# Defines

In [ ]:
input_data_dir = os.path.join("data", "processed")
cur_file_path = os.path.join(input_data_dir, "tr_fincrime_train.parquet")

cache_dir = "cache"
os.makedirs(cache_dir, exist_ok=True)

In [ ]:
url = 'https://www2.census.gov/geo/tiger/GENZ2018/shp/cb_2018_us_state_500k.zip'
response = requests.get(url)
with zipfile.ZipFile(io.BytesIO(response.content)) as z:
    z.extractall(os.path.join(cache_dir, 'states_shp'))

# Data

In [ ]:
df = pd.read_parquet(cur_file_path)

In [ ]:
df["distance"] = np.linalg.norm(
    df[["lat", "long"]].values - df[["merch_lat", "merch_long"]].values,
    ord=2,
    axis=1
)

In [ ]:
df["age_at_transaction"] = (df["trans_date_trans_time"] - df["dob"]) / pd.Timedelta("365d")

In [ ]:
df["trans_hour"] = df["trans_date_trans_time"].dt.hour

# Data Familiarisation

## Raw Data

- No missing values

In [ ]:
df.head()

In [ ]:
df.info()

## Date Ranges

- Transactions include the onset of COVID pandemic
- No obvious outliers in `dob` field

In [ ]:
df[DataProcessor.date_cols].describe()

## Individuals

983 cards and 693 merchants

In [ ]:
df[["cc_num", "merchant"]].nunique()

No individual has more than 1 credit card

In [ ]:
df.groupby(["first", "last", "gender", "street", "zip"])["cc_num"].nunique().max()

## Location

- Card holders are based in the US
- 

In [ ]:
df_plot = df.groupby("state").size().sort_values(ascending=False).to_frame("Count")
df_plot["Percentage"] = df_plot["Count"] / df_plot["Count"].sum()
df_plot.head(15)

In [ ]:
usa = gpd.read_file(os.path.join(cache_dir, 'states_shp/cb_2018_us_state_500k.shp'))
merged = usa.merge(df_plot, left_on="STUSPS", right_index=True)
merged = merged.sort_values(by='Count', ascending=False)

fig, ax = plt.subplots(1, 1, figsize=(10, 10))
merged.plot(
    column='Count',
    cmap='OrRd',
    linewidth=0.8,
    ax=ax,
    edgecolor='0.8',
    legend=True
)
# Remove axis off
ax.axis('off')
ax.set_xlim([-130, -65])  
ax.set_ylim([24, 50])

# Add a title
ax.set_title('Counts per State of Residence', fontdict={'fontsize': '15', 'fontweight' : '3'})

merged['coords'] = merged['geometry'].apply(lambda x: x.representative_point().coords[:])
merged['coords'] = [coords[0] for coords in merged['coords']]
for idx, row in merged.head(10).iterrows():
    plt.annotate(
        text=f"{row['STUSPS']}: {round(row['Count']/10_000, 2) }k", 
        xy=row['coords'],
        horizontalalignment='center', 
        fontsize=9
    )

for idx, row in merged.tail(10).iterrows():
    plt.annotate(
        text=f"{row['STUSPS']}: {round(row['Count']/10_000, 2) }k", 
        xy=row['coords'],
        horizontalalignment='center', 
        fontsize=9
    )

plt.show()

# Fraud Stats

## Fraud Counts
- Highly imbalanced (0.5% - or 7506 out of 1296675 - are fraudulent)
- Cases of fraud do not happen at a constant rate.

In [ ]:
df["is_fraud"].describe(), df["is_fraud"].sum()

In [ ]:
df_plot = df.groupby('cc_num')['is_fraud'].agg(['count', 'sum'])

df_plot = df_plot.rename(columns={
    "count": "Number of Transactions", 
    "sum": "Number of Fraudulent Transactions"
})

sns.scatterplot(
    df_plot,
    x='Number of Transactions',
    y='Number of Fraudulent Transactions'
)

## Transaction amounts
- Fraudulent transactions tend to have higher transaction amounts on average, but they were not the largest outright
- Half of all fraudulent transactions were $\approx$$400 or above; most transactions at this level are normal though

In [ ]:
df.groupby("is_fraud")["amt"].describe()

In [ ]:
sns.histplot(
    data=df,
    x="amt",
    hue="is_fraud", 
    stat="percent",
    log_scale=(True, False),
    common_norm=False,
    fill=True,
    bins=100
)

plt.gca().xaxis.set_major_formatter(ticker.ScalarFormatter())
plt.gca().xaxis.set_major_locator(ticker.LogLocator(base=10.0))

plt.title('Transaction Proportions by Amount: Fraud vs Non-Fraud \n Percentages for Each Group')

In [ ]:
sns.histplot(
    data=df.loc[df["amt"] > 400],
    x="amt",
    hue="is_fraud", 
    stat="percent",
    log_scale=(True, False),
    common_norm=True,
    fill=True,
    bins=200
)

plt.gca().xaxis.set_major_formatter(ticker.ScalarFormatter())
plt.gca().xaxis.set_major_locator(ticker.LogLocator(base=10.0))

plt.title('Transactions over $400 by Amount: Fraud vs Non-Fraud \n Percentages for All Data')

## Time of Day

- Fraudulent transactions are more prevalent during the late hours between 22:00 and 03:00.

In [ ]:
np.array([70, 86]) * .82

In [ ]:
# df_plot = df.groupby(["trans_hour", "category"])["is_fraud"].mean().to_frame("prop") / df["is_fraud"].mean()
df_plot = df.groupby(["trans_hour"])["is_fraud"].mean().to_frame("odds") / df["is_fraud"].mean()
df_plot = df_plot.reset_index()
df_plot = df_plot.rename(columns={
    "trans_hour": "Hour",
    "odds": "Relative Fraud Odds"
})

sns.barplot(
    data=df_plot,
    x='Hour',
    y='Relative Fraud Odds',
)

plt.a

## Merchant Category

### Total Counts

In [ ]:
df_plot = df["category"].value_counts(dropna=False).to_frame("Counts")
df_plot = df_plot.reset_index()
df_plot = df_plot.rename(columns={"category": "Merchant Category"})

In [ ]:
plt.figure(figsize=(20, 6))

sns.barplot(
    data=df_plot.reset_index(),
    x='Merchant Category',
    y='Counts',
)

### Fraud Odds vs Population Baseline

How much more likely is a transaction to be fraudulent vs the population baseline - i.e. ${\displaystyle \frac{P(\text{fraud} | \text{Merchant Category}=x )}{P(\text{fraud} )}}$

- Internet shopping and internet misc have the highest relative odds (at least 2.5 more likely)
- PoS grocery also have higher odds

In [ ]:
df_plot = df.groupby("category")["is_fraud"].mean().sort_values(ascending=False).to_frame("Fraud Odds") / df["is_fraud"].mean()
df_plot = df_plot.reset_index()
df_plot = df_plot.rename(columns={"category": "Merchant Category"})

In [ ]:
df_plot

In [ ]:
plt.figure(figsize=(20, 6))

sns.barplot(
    data=df_plot.reset_index(),
    x='Merchant Category',
    y='Fraud Odds',
)

## Merchant

- Almost every merchant in the dataset had at least 1 fraudulent transaction reported against them
- There are some differences between the merchant GPS coordinates of fraudulent and non-fraudulent transactions
- 

In [ ]:
df_loc = df.groupby(['merchant', "is_fraud"])[['merch_lat', 'merch_long']].agg(["count", "mean", "std"])
df_loc = df_loc.fillna(0)
# df_log = df_loc.join(
#     df.groupby(['merchant', "is_fraud"]).size().to_frame("count")
# )

# df_loc_mean = df.groupby(['merchant', "is_fraud"])[['merch_lat', 'merch_long']].mean()
# df_loc_std = df.groupby(['merchant', "is_fraud"])[['merch_lat', 'merch_long']].std().fillna(0)
# df_loc_std = df_loc_std.join(
#     df.groupby(['merchant', "is_fraud"]).size().to_frame("count")
# )

In [ ]:
df.groupby(["merchant", "merch_lat", "merch_long"])["is_fraud"].mean()

In [ ]:
merch_selection = df_loc[
    df_loc[("merch_lat", "std")] == 0
].index.unique("merchant")

In [ ]:
df_loc.loc[
    idx[merch_selection, :]
].head(20)

In [ ]:
df_loc_std[(df_loc_std==0).any(axis=1)]

In [ ]:
df_loc.loc[:, idx[:, "std"]].sort_values(by=[("merch_lat", "std")])

In [ ]:
df.groupby('merchant')[['merch_lat', 'merch_long']].std()

In [ ]:
df_count = df.groupby('merchant')['is_fraud'].mean()

In [ ]:
sum(df_count != 0)

In [ ]:
df.groupby('merchant')['is_fraud'].mean().sort_values(ascending=False).head(200)

In [ ]:
df.loc[
    df['merchant'] == 'fraud_Kozey-Boehm'
]